In [1]:
from pysr import PySRRegressor

from typing import Any
from collections import defaultdict
import warnings
import time
import pickle
import os

import torch
import numpy as np

from flash_ansr import FlashANSRDataset
from flash_ansr.utils import load_config, substitute_root_path, get_path

import simplipy
from simplipy.utils import numbers_to_constant

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
def remove_padding(x: torch.Tensor | np.ndarray, skeleton: list[str], variables: set[str]) -> torch.Tensor | np.ndarray:
    # Delete all columns corresponding to unused variables
    used_vars_mask = [var in skeleton for var in variables]
    return x[:, used_vars_mask]


def create_model(timeout_in_seconds: int, niterations: int, use_mult_div_operators: bool) -> PySRRegressor:
    if use_mult_div_operators:
        additional_unary_operators = [
            'mult2(x) = 2*x',
            'mult3(x) = 3*x',
            'mult4(x) = 4*x',
            'mult5(x) = 5*x',
            'div2(x) = x/2',
            'div3(x) = x/3',
            'div4(x) = x/4',
            'div5(x) = x/5']
        additional_extra_sympy_mappings = {
            "mult2": simplipy.operators.mult2,
            "mult3": simplipy.operators.mult3,
            "mult4": simplipy.operators.mult4,
            "mult5": simplipy.operators.mult5,
            "div2": simplipy.operators.div2,
            "div3": simplipy.operators.div3,
            "div4": simplipy.operators.div4,
            "div5": simplipy.operators.div5,
        }
    else:
        additional_unary_operators = []
        additional_extra_sympy_mappings = {}

    print(f'Creating PySR model with timeout {timeout_in_seconds=}, {niterations=}, and {use_mult_div_operators=} ...')

    return PySRRegressor(
        temp_equation_file=True,
        delete_tempfiles=True,
        timeout_in_seconds=timeout_in_seconds,
        niterations=niterations,
        unary_operators=[
            'neg',
            'abs',
            'inv',
            'sin',
            'cos',
            'tan',
            'asin',
            'acos',
            'atan',
            'exp',
            'log',
            'pow2(x) = x^2',
            'pow3(x) = x^3',
            'pow4(x) = x^4',
            'pow5(x) = x^5',
            r'pow1_2(x::T) where {T} = x >= 0 ? T(x^(1/2)) : T(NaN)',
            r'pow1_3(x::T) where {T} = x >= 0 ? T(x^(1/3)) : T(-((-x)^(1/3)))',
            r'pow1_4(x::T) where {T} = x >= 0 ? T(x^(1/4)) : T(NaN)',
            r'pow1_5(x::T) where {T} = x >= 0 ? T(x^(1/5)) : T(-((-x)^(1/5)))',
        ] + additional_unary_operators,
        binary_operators=['+', '-', '*', '/', '^'],
        extra_sympy_mappings={
            "pow2": simplipy.operators.pow2,
            "pow3": simplipy.operators.pow3,
            "pow4": simplipy.operators.pow4,
            "pow5": simplipy.operators.pow5,
            "pow1_2": simplipy.operators.pow1_2,
            "pow1_3": lambda x: x**(1 / 3),  # Workaround for https://stackoverflow.com/questions/68577498/sympy-typeerror-cannot-determine-truth-value-of-relational-how-to-make-sure-x
            "pow1_4": simplipy.operators.pow1_4,
            "pow1_5": lambda x: x**(1 / 5),
        } | additional_extra_sympy_mappings,
        constraints={
            '^': (-1, 3)
        },
    )

In [8]:
n_support = 512
max_n_support = 1024
size = 100
verbose = True
timeout_in_seconds = 5
niterations = 128
use_mult_div_operators = False
noise_level = 0
parsimony = None
padding = False

dataset = FlashANSRDataset.from_config(get_path('data', 'ansr-data', 'test_set', 'feynman', 'dataset.yaml'))

collected = 0
iterator = dataset.iterate(
    size=size * 2,  # In case something goes wrong in a few samples, we have enough buffer to still collect 'size' samples
    max_n_support=max_n_support,
    n_support=n_support * 2,
    verbose=verbose,
    batch_size=1,
    tqdm_kwargs={'desc': 'Evaluating', 'total': size},
    tokenizer_oov='unk'  # Do not raise an error if an unknown token (operator) is encountered
)

model = create_model(
    timeout_in_seconds=timeout_in_seconds,
    niterations=niterations,
    use_mult_div_operators=use_mult_div_operators,
)

results_dict = defaultdict(list)

Compiling Skeletons: 100%|██████████| 89/89 [00:00<00:00, 20847.37it/s]


Creating PySR model with timeout timeout_in_seconds=5, niterations=128, and use_mult_div_operators=False ...


In [ ]:
for batch_id, batch in enumerate(iterator):
    batch = dataset.collate(batch, device='cpu')
    print(batch['expression'][0])
    print(batch['x_tensors'][0][0])
    print()

Evaluating: 200it [00:00, 1184.14it/s]                       


['*', '1.5938927', '*', 'x1', '*', 'pow2', 'x2', '/', 'x3', '/', 'x4', 'x5']
tensor([11.9617, -9.0395,  4.8197, -0.2114,  7.9646,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000])
['pow1_2', '+', 'pow2', '-', 'x1', 'x2', 'pow2', '-', 'x3', 'x4']
tensor([ -0.4883,   0.4816,   3.1846, -11.2533,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000])
['/', '1.1654137', '*', 'x1', 'x2']
tensor([ -0.1456, -21.9542,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000])
['/', 'x1', '-', '1.9311517', '/', 'x2', 'x3']
tensor([-5.4296,  3.0952,  6.2766,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000])
['/', '3.1331372', '*', 'x1', '+', 'pow2', 'x2', '+', 'pow2', 'x3', 'pow2', 'x4']
tensor([-1.3770,  4.8671, -0.6842, -1.5387,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000])
['*', 'x1', '*', 'pow2', 'x2', '*', '/', '9.217493', '*', 'x3', '*', 'x4', 'pow2', 'x5', '/', 'pow4', 'x6', 'pow2', '-', 'pow2', 'x6', 'p

In [4]:
if verbose:
    print(f'Starting evaluation on {size} problems...')

for batch_id, batch in enumerate(iterator):
    batch = dataset.collate(batch, device='cpu')

    if n_support is None:
        n_support = batch['x_tensors'].shape[1] // 2

    if n_support == 0:
        warnings.warn('n_support evaluated to zero. Skipping batch.')
        continue

    if noise_level > 0.0:
        batch['y_tensors_noisy'] = batch['y_tensors'] + (
            noise_level * batch['y_tensors'].std() * torch.randn_like(batch['y_tensors'])
        )
        if not torch.all(torch.isfinite(batch['y_tensors_noisy'])):
            warnings.warn('Adding noise to the target variable resulted in non-finite values. Skipping this sample.')
            continue
    else:
        batch['y_tensors_noisy'] = batch['y_tensors']

    x_numpy = batch['x_tensors'].cpu().numpy()[0]
    y_numpy = batch['y_tensors'].cpu().numpy()[0]
    y_noisy_numpy = batch['y_tensors_noisy'].cpu().numpy()[0]

    X = x_numpy[:n_support]
    y = y_noisy_numpy[:n_support]

    X_val = x_numpy[n_support:]
    y_val = y_noisy_numpy[n_support:]

    print(batch['expression'][0])

    sample_results = {
        'skeleton': batch['skeleton'][0],
        'skeleton_hash': batch['skeleton_hash'][0],
        'expression': batch['expression'][0],
        'input_ids': batch['input_ids'][0].cpu().numpy(),
        'labels': batch['labels'][0].cpu().numpy(),
        'constants': [c.cpu().numpy() for c in batch['constants'][0]],
        'x': X,
        'y': y_numpy[:n_support],
        'y_noisy': y,
        'x_val': X_val,
        'y_val': y_numpy[n_support:],
        'y_noisy_val': y_val,
        'n_support': n_support,
        'labels_decoded': dataset.tokenizer.decode(batch['labels'][0].cpu().tolist(), special_tokens='<constant>'),
        'parsimony': parsimony,

        'fit_time': None,
        'predicted_expression': None,
        'predicted_expression_prefix': None,
        'predicted_skeleton_prefix': None,
        'predicted_constants': None,
        'predicted_score': None,
        'predicted_log_prob': None,
        'y_pred': None,
        'y_pred_val': None,
        'prediction_success': False,
        'error': None,
    }

    error_occured = False

    print(X[0])

    if not padding:
        used_variables = [variable for variable in dataset.skeleton_pool.variables if variable in sample_results['skeleton']]  # Keep order
        X = remove_padding(X, sample_results['skeleton'], dataset.skeleton_pool.variables)
        X_val = remove_padding(X_val, sample_results['skeleton'], dataset.skeleton_pool.variables)
    else:
        used_variables = None

    
    print(f'Used variables: {used_variables}')
    print(X[0])

    fit_time_before = time.time()
    try:
        model.fit(X, y, variable_names=used_variables)
        sample_results['fit_time'] = time.time() - fit_time_before
        sample_results['prediction_success'] = True
    except Exception as e:
        error_occured = True
        sample_results['error'] = str(e)

    if not error_occured:
        y_pred = model.predict(X).reshape(-1, 1)
        y_pred_val = model.predict(X_val).reshape(-1, 1)

        if not y_pred.shape == y.shape:
            raise ValueError(f"Shape of y_pred {y_pred.shape} does not match shape of y {y.shape}.")
        if not y_pred_val.shape == y_val.shape:
            raise ValueError(f"Shape of y_pred_val {y_pred_val.shape} does not match shape of y_val {y_val.shape}.")

        sample_results['y_pred'] = y_pred
        sample_results['y_pred_val'] = y_pred_val

        predicted_expression = str(model.get_best()['equation'])
        sample_results['predicted_expression'] = predicted_expression
        sample_results['predicted_expression_prefix'] = dataset.simplipy_engine.infix_to_prefix(predicted_expression)
        sample_results['predicted_skeleton_prefix'] = numbers_to_constant(sample_results['predicted_expression_prefix'])

    for key, value in sample_results.items():
        results_dict[key].append(value)

    if not len(set(len(v) for v in results_dict.values())) == 1:
        print({k: len(v) for k, v in results_dict.items()})  # Check that all lists have the same length
        raise ValueError("Not all lists in results_dict have the same length.")
    
    break

Starting evaluation on 1 problems...


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]/home/psaegert/miniconda3/envs/flash-ansr-compat/lib/python3.13/site-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...


['/', '2.5230684', '*', 'x1', '*', '-', 'x2', '-4.1478057', '/', 'x3', 'x4']
[ 9.036314   -0.73590845  7.409702   16.52647     0.          0.
  0.          0.          0.          0.        ]
Used variables: ['x1', 'x2', 'x3', 'x4']
[ 9.036314   -0.73590845  7.409702   16.52647   ]


┌ Warning: You have passed over 15 unary operators. To prevent long compilation times, some optimizations will be disabled. If this presents an issue, please open an issue on https://github.com/SymbolicML/DynamicExpressions.jl
└ @ DynamicExpressions.OperatorEnumConstructionModule ~/.julia/packages/DynamicExpressions/cYbpm/src/OperatorEnumConstruction.jl:446
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:



Expressions evaluated per second: 2.230e+05
Progress: 1117 / 3968 total iterations (28.150%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.599e+02  0.000e+00  y = 0.5045
2           4.098e+01  1.847e+00  y = inv(x3)
3           3.219e+01  2.414e-01  y = 0.83585 / x3
5           8.094e+00  6.903e-01  y = exp(neg(x2)) / x3
6           4.302e+00  6.320e-01  y = (0.89147 / x3) / exp(x2)
7           4.151e+00  3.574e-02  y = (exp(neg(x2)) + -0.10993) / x3
8           3.956e+00  4.826e-02  y = (0.86695 / x3) / (exp(x2) + -0.015619)
10          3.741e+00  2.782e-02  y = ((exp(x2 * -1.3163) + 0.75292) / x3) / 2.0264
11          3.220e+00  1.502e-01  y = ((exp(neg(x2)) + -0.10997) / x3) + (0.14588 / x1)
13          3.208e+00  1.754e-03  y = (0.14508 / x1) + (((exp(neg(x2)) + -0.11025) 

In [5]:
print(results_dict['predicted_expression_prefix'][0])
print(results_dict['y'][0][:10] - results_dict['y_pred'][0][:10])
print()

['/', '/', '/', '-', 'inv', 'x1', '/', 'neg', '15.71321', 'x3', 'neg', '2.0698884', '+', 'x2', '2.2714472', 'neg', '3.9333205']
[[ 0.00404464]
 [-0.00634118]
 [-0.32136279]
 [ 0.08063915]
 [ 0.01004292]
 [ 0.00680243]
 [ 0.01676378]
 [ 1.96853852]
 [-0.0455566 ]
 [ 0.02516805]]

